In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24939
2,Huelva,Confirmados PDIA 14 días,330
3,Huelva,Tasa PDIA 14 días,"64,30617534150477"
4,Huelva,Confirmados PDIA 7 días,199
5,Huelva,Total Confirmados,25140
6,Huelva,Curados,18661
7,Huelva,Fallecidos,343


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24939.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6613.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4110 personas en los últimos 7 días 

Un positivo PCR cada 2664 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24939.0,199.0,330.0,513170.0,38.778572,64.306175,109.0
Condado-Campiña,7035.0,95.0,148.0,156231.0,60.807394,94.731519,55.0
Huelva-Costa,15185.0,96.0,173.0,289548.0,33.155125,59.748297,49.0
Huelva (capital),6613.0,35.0,54.0,143837.0,24.333099,37.542496,17.0
Almonte,894.0,19.0,30.0,24507.0,77.528869,122.414004,14.0
Bollullos Par del Condado,629.0,11.0,18.0,14387.0,76.457913,125.112949,8.0
Cartaya,971.0,14.0,23.0,20083.0,69.710701,114.524722,8.0
Isla Cristina,1997.0,9.0,16.0,21393.0,42.069836,74.790819,7.0
Moguer,844.0,10.0,15.0,21867.0,45.731010,68.596515,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,108.0,10.0,21.0,3092.0,323.415265,679.172057,6.0
San Bartolomé de la Torre,209.0,9.0,16.0,3761.0,239.298059,425.418772,3.0
San Silvestre de Guzmán,13.0,1.0,2.0,614.0,162.866450,325.732899,0.0
Villalba del Alcor,455.0,9.0,10.0,3366.0,267.379679,297.088532,4.0
Lucena del Puerto,161.0,1.0,8.0,3261.0,30.665440,245.323520,-1.0
Corteconcepción,24.0,1.0,1.0,536.0,186.567164,186.567164,0.0
Ayamonte,1231.0,12.0,35.0,21104.0,56.861259,165.845337,4.0
Villarrasa,132.0,3.0,3.0,2211.0,135.685210,135.685210,2.0
Bollullos Par del Condado,629.0,11.0,18.0,14387.0,76.457913,125.112949,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,161.0,1.0,8.0,3261.0,30.665440,245.323520,-1.0,0.125000
San Juan del Puerto,365.0,2.0,7.0,9411.0,21.251727,74.381043,0.0,0.285714
Ayamonte,1231.0,12.0,35.0,21104.0,56.861259,165.845337,4.0,0.342857
Puebla de Guzmán,108.0,10.0,21.0,3092.0,323.415265,679.172057,6.0,0.476190
San Silvestre de Guzmán,13.0,1.0,2.0,614.0,162.866450,325.732899,0.0,0.500000
Aljaraque,926.0,1.0,2.0,21474.0,4.656794,9.313589,0.0,0.500000
Beas,163.0,1.0,2.0,4341.0,23.036167,46.072334,1.0,0.500000
Alosno,124.0,2.0,4.0,3933.0,50.851767,101.703534,1.0,0.500000
Huelva-Costa,15185.0,96.0,173.0,289548.0,33.155125,59.748297,49.0,0.554913
